In [2]:
!pip install simpletransformers
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 633.7 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 3.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 36.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 66.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.1 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=b3fd438248c1a8d9a774609de59641a27d2a4dd4ad36fe50a63f8c43d51b14b5
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [3]:
import numpy as np 
import pandas as pd 
import os
import wandb
import json
import logging
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

2024-04-12 06:33:55.533812: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-12 06:33:55.533912: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-12 06:33:55.632569: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/stanford-question-answering-dataset/train-v1.1.json
/kaggle/input/stanford-question-answering-dataset/dev-v1.1.json


In [4]:
with open('/kaggle/input/stanford-question-answering-dataset/train-v1.1.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [5]:
converted_data = []

for item in data['data']:
    context = item['paragraphs'][0]['context']
    qas = item['paragraphs'][0]['qas']
    
    qa_list = []
    
    for qa in qas:
        question = qa['question']
        if "what" in question.lower():
            question = "what"
        elif "who" in question.lower():
            question = "who"
        elif "where" in question.lower():
            question = "where"
        elif "when" in question.lower():
            question = "when"
        elif "how" in question.lower():
            question = "how"
        elif "which" in question.lower():
            question = "which"
        
        
        answers = qa['answers']
        if answers:
            answer_text = answers[0]['text']
            answer_start = answers[0]['answer_start']
            qa_id = qa['id']
            
            qa_dict = {
                'id': qa_id,
                'is_impossible': False,
                'question': question,
                'answers': [
                    {
                        'text': answer_text,
                        'answer_start': answer_start,
                    }
                ],
            }
            
            qa_list.append(qa_dict)
    
    context_dict = {
        'context': context,
        'qas': qa_list,
    }
    
    converted_data.append(context_dict)

print(json.dumps(converted_data, indent=4))

[
    {
        "context": "Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend \"Venite Ad Me Omnes\". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",
        "qas": [
            {
                "id": "5733be284776f41900661182",
                "is_impossible": false,
                "question": "who",
                "answers": [
                    {
                        "text": "Saint Bernadette Soubirous",
   

In [6]:
with open('converted_data.json', 'w', encoding='utf-8') as outfile:
     json.dump(converted_data, outfile, indent=4)

In [7]:
with open('/kaggle/input/stanford-question-answering-dataset/dev-v1.1.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)

In [8]:
converted_data_test = []

for item in data['data']:
    context = item['paragraphs'][0]['context']
    qas = item['paragraphs'][0]['qas']
    
    qa_list = []
    
    for qa in qas:
        question = qa['question']
        if "what" in question.lower():
            question = "what"
        elif "who" in question.lower():
            question = "who"
        elif "where" in question.lower():
            question = "where"
        elif "when" in question.lower():
            question = "when"
        elif "how" in question.lower():
            question = "how"
        elif "which" in question.lower():
            question = "which"
            
        answers = qa['answers']
        if answers:
            answer_text = answers[0]['text']
            answer_start = answers[0]['answer_start']
            qa_id = qa['id']
            
            qa_dict = {
                'id': qa_id,
                'is_impossible': False,  
                'question': question,
                'answers': [
                    {
                        'text': answer_text,
                        'answer_start': answer_start,
                    }
                ],
            }
            
            qa_list.append(qa_dict)
    
    context_dict = {
        'context': context,
        'qas': qa_list,
    }
    
    converted_data_test.append(context_dict)

print(json.dumps(converted_data_test, indent=4))

[
    {
        "context": "Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend \"Venite Ad Me Omnes\". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",
        "qas": [
            {
                "id": "5733be284776f41900661182",
                "is_impossible": false,
                "question": "who",
                "answers": [
                    {
                        "text": "Saint Bernadette Soubirous",
   

In [9]:
import random
import json

converted_validation_data = []

eval_data_subset = random.sample(converted_data, k=100)

for item in eval_data_subset:
    context = item['context']
    qas = item['qas']
    
    qa_list = []
    
    for qa in qas:
        question = qa['question']
        if "what" in question.lower():
            question = "what"
        elif "who" in question.lower():
            question = "who"
        elif "where" in question.lower():
            question = "where"
        elif "when" in question.lower():
            question = "when"
        elif "how" in question.lower():
            question = "how"
        elif "which" in question.lower():
            question = "which"
            
        answers = qa['answers']
        if answers:
            answer_text = answers[0]['text']
            answer_start = answers[0]['answer_start']
            qa_id = qa['id']
            is_impossible = qa['is_impossible']
            
            qa_dict = {
                'id': qa_id,
                'is_impossible': is_impossible,
                'question': question,
                'answers': [
                    {
                        'text': answer_text,
                        'answer_start': answer_start,
                    }
                ],
            }
            
            qa_list.append(qa_dict)
    
    context_dict = {
        'context': context,
        'qas': qa_list,
    }
    
    converted_validation_data.append(context_dict)

validation_data = json.dumps(converted_validation_data, ensure_ascii=False, indent=4)

print(validation_data)


[
    {
        "context": "Strasbourg (/ˈstræzbɜːrɡ/, French pronunciation: ​[stʁaz.buʁ, stʁas.buʁ]; Alsatian: Strossburi; German: Straßburg, [ˈʃtʁaːsbʊɐ̯k]) is the capital and largest city of the Alsace-Champagne-Ardenne-Lorraine (ACAL) region in eastern France and is the official seat of the European Parliament. Located close to the border with Germany, it is the capital of the Bas-Rhin département. The city and the region of Alsace were historically predominantly Alemannic-speaking, hence the city's Germanic name. In 2013, the city proper had 275,718 inhabitants, Eurométropole de Strasbourg (Greater Strasbourg) had 475,934 inhabitants and the Arrondissement of Strasbourg had 482,384 inhabitants. With a population of 768,868 in 2012, Strasbourg's metropolitan area (only the part of the metropolitan area on French territory) is the ninth largest in France and home to 13% of the ACAL region's inhabitants. The transnational Eurodistrict Strasbourg-Ortenau had a population of 915,000 in

In [10]:
import random
import json

eval_data_subset = random.sample(converted_data, k=100)

output_file = 'random_validation_data.json'
with open(output_file, 'w', encoding='utf-8') as outfile:
    json.dump(eval_data_subset, outfile, ensure_ascii=False, indent=4)

print(f"Randomly sampled validation data saved to '{output_file}'.")

Randomly sampled validation data saved to 'random_validation_data.json'.


In [11]:
model_type="bert"
model_name= "bert-base-cased"

In [12]:
wandb.init(project="Question Answer Application")

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [13]:
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size = 3
model_args.num_train_epochs = 5

In [15]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args,use_cuda=True,
)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [23]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    'num_train_epochs': 10,
    'learning_rate': 1e-6,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size": 3,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    "use_gpu": True,  # Use GPU for training
    "fp16": True,  # Use mixed-precision training
    "fp16_opt_level": "O1",  # Mixed-precision optimization level
    "gradient_accumulation_steps": 2,  # Accumulate gradients to simulate larger batch size
}

In [ ]:
model.train_model(converted_data, eval_data='random_validation_data.json')

add example index and unique id: 100%|██████████| 2596/2596 [00:00<00:00, 614782.52it/s]


Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Training loss,█▄▂▂▁▁
correct,▁▁▂▁▁▁▁▂▂▂▃▃▄▄▄▅▅▆▆▆▇▇▇▇██████
eval_loss,███▇▇▇▆▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁
global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
incorrect,██▄▁▁▁▂▃▃▃▄▄▄▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
lr,█▇▅▄▂▁
similar,▁▁▅███▇▆▆▅▅▄▄▃▄▃▃▃▃▃▃▃▃▃▂▂▃▃▃▃
train_loss,█▇▆▆▅▄▄▄▄▃▃▄▃▄▃▂▂▃▂▂▃▂▂▂▂▂▃▁▁▂
Training loss,2.45068
correct,94
eval_loss,-2.58832


Running Epoch 1 of 30:   0%|          | 0/21 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/9 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/9 [00:00<?, ?it/s]

Running Epoch 17 of 30:   0%|          | 0/21 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/9 [00:00<?, ?it/s]

Running Epoch 18 of 30:   0%|          | 0/21 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/9 [00:00<?, ?it/s]

In [17]:
to_predict = [
   {
    "context": "Sachin tendulkar is a legendary cricketer born in Mumbai in 1979. He was great in his prime.",
    "qas": [
        {
            "id": "00001",
            "question": "who",
            "answers": [
                {
                    "text": "",
                    "answer_start": -1
                }
            ]
        }
    ]
}
]
model.predict(to_predict=to_predict)


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 3160.74it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'id': '00001', 'answer': ['1979', 'Mumbai in 1979']}],
 [{'id': '00001', 'probability': [0.6675499808239992, 0.23942103149228125]}])

In [18]:
to_predict = [
   {
    "context": "Sachin tendulkar is a legendary cricketer born in Mumbai in 1979. He was great in his prime.",
    "qas": [
        {
            "id": "00001",
            "question": "what",
            "answers": [
                {
                    "text": "",
                    "answer_start": -1
                }
            ]
        }
    ]
}
]
model.predict(to_predict=to_predict)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 3539.50it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'id': '00001', 'answer': ['1979', 'Mumbai in 1979', 'Mumbai']}],
 [{'id': '00001',
   'probability': [0.590656146012179,
    0.2720112445901566,
    0.11163318985271471]}])

In [19]:
to_predict = [
   {
    "context": "Sachin tendulkar is a legendary cricketer born in Mumbai in 1979. He was great in his prime.",
    "qas": [
        {
            "id": "00001",
            "question": "where",
            "answers": [
                {
                    "text": "",
                    "answer_start": -1
                }
            ]
        }
    ]
}
]
model.predict(to_predict=to_predict)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1399.50it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'id': '00001', 'answer': ['1979', 'Mumbai in 1979', 'Mumbai']}],
 [{'id': '00001',
   'probability': [0.5167102624760819,
    0.2803823399858366,
    0.11872135249590543]}])

In [20]:
to_predict = [
   {
    "context": "Sachin tendulkar is a legendary cricketer born in Mumbai in 1979. He was great in his prime.",
    "qas": [
        {
            "id": "00001",
            "question": "when",
            "answers": [
                {
                    "text": "",
                    "answer_start": -1
                }
            ]
        }
    ]
}
]
model.predict(to_predict=to_predict)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 4198.50it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'id': '00001', 'answer': ['1979', 'Mumbai in 1979']}],
 [{'id': '00001', 'probability': [0.7300707989302742, 0.13852241300360704]}])

In [21]:
to_predict = [
   {
    "context": "Sachin tendulkar is a legendary cricketer born in Mumbai in 1979. He was great in his prime.",
    "qas": [
        {
            "id": "00001",
            "question": "which",
            "answers": [
                {
                    "text": "",
                    "answer_start": -1
                }
            ]
        }
    ]
}
]
model.predict(to_predict=to_predict)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1691.93it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'id': '00001', 'answer': ['1979', 'Mumbai in 1979', 'Mumbai']}],
 [{'id': '00001',
   'probability': [0.5501479775390644,
    0.2543478751286076,
    0.1015685730692078]}])

In [22]:
to_predict = [
   {
    "context": "Sachin tendulkar is a legendary cricketer born in Mumbai in 1979. He was great in his prime.",
    "qas": [
        {
            "id": "00001",
            "question": "how",
            "answers": [
                {
                    "text": "",
                    "answer_start": -1
                }
            ]
        }
    ]
}
]
model.predict(to_predict=to_predict)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1546.57it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'id': '00001', 'answer': ['1979', 'Mumbai in 1979']}],
 [{'id': '00001', 'probability': [0.8185501055748271, 0.1384059261511695]}])

In [ ]:
from transformers import QuestionAnsweringPipeline, AutoModelForQuestionAnswering, AutoTokenizer

# Initialize the pipeline
model_checkpoint = f"{model_type}-base-cased"  # Adjust based on your model type
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
qa_pipeline = QuestionAnsweringPipeline(model=model, tokenizer=tokenizer)

# Save the model
output_dir = f"outputs/{model_type}/trained_model"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Save the pipeline for later use
qa_pipeline.save_pretrained(output_dir)

print(f"Trained model saved at: {output_dir}")

In [ ]:
"""output_dir = model.args.output_dir

# Save the model using the output directory
model.save_model(output_dir)"""

In [ ]:
import torch
from simpletransformers.question_answering import QuestionAnsweringModel
from transformers import BertTokenizer

# Load the saved model and tokenizer
saved_model_dir = "outputs/bert/trained_model"  # Adjust based on your saved model directory
model = QuestionAnsweringModel("bert", saved_model_dir)

# Create a tokenizer object
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

# Define the context and question for which you want to generate predictions
context = "Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary."
question = "what"

# Tokenize the input
inputs = tokenizer(question, context, return_tensors="pt")

# Generate predictions
outputs = model.model(**inputs)

# Decode the predicted answer
answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits) + 1
answer = tokenizer.decode(inputs["input_ids"][0][answer_start:answer_end])

# Print the predicted answer
print("Predicted Answer:", answer)


In [ ]:
context = "Sachin tendulkar is a legendary cricketer born in Mumbai in 1979. He was great in his prime."
question = "who"

# Tokenize the input
inputs = tokenizer(question, context, return_tensors="pt")

# Generate predictions
outputs = model.model(**inputs)

# Decode the predicted answer
answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits) + 1
answer = tokenizer.decode(inputs["input_ids"][0][answer_start:answer_end])

# Print the predicted answer
print("Predicted Answer:", answer)


# Full fledged